In [40]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.parsing.preprocessing import strip_short, remove_stopwords, strip_non_alphanum, strip_numeric, strip_multiple_whitespaces, strip_punctuation, strip_tags
%matplotlib inline

In [41]:
# Download data
#import requests
#request = requests.get("https://drive.google.com/uc?export=download&id=1wHt8PsMLsfX5yNSqrt2fSTcb8LEiclcf")
#with open("data.zip", "wb") as file:
#    file.write(request.content)

# Unzip data
#import zipfile
#with zipfile.ZipFile('data.zip') as zip:
#    zip.extractall('data')

In [42]:
 # Load data and set labels
#data_complaint = pd.read_csv('data/complaint1700.csv')
#data_complaint['label'] = 0
#data_non_complaint = pd.read_csv('data/noncomplaint1700.csv')
#data_non_complaint['label'] = 1

# Concatenate complaining and non-complaining data
#data = pd.concat([data_complaint, data_non_complaint], axis=0).reset_index(drop=True)

# Drop 'airline' column
#data.drop(['airline'], inplace=True, axis=1)

# Display 5 random samples
#data.sample(5)

# LOAD TWEET DATA

In [43]:
data = pd.read_csv('data/auto_tweets.csv')
#data = pd.read_csv('./air_plane_tweets.csv')
data = data.sample(frac = 1) 


In [44]:
data['auto_sentiment'] = data['auto_sentiment'].map(lambda x: x+1)

In [45]:
test_data = data[:100000]
data = data[100000:1000000]

#test_data = test_data[:100]
#data = data[:900]


In [46]:
print(len(data))
print(len(test_data))

900000
100000


In [47]:
#data= data[:1000]
data.sample(10)


,text,sentiment,auto_sentiment
666344,@mellemusic clicking clicking - nothing happe...,0,1
589478,hurray for texting!!!!! but youu can't hear c...,0,0
1044506,looking forward to a relaxing night,1,2
1300610,"@manroo Wow, Fabio Capello has the worst luck ...",1,2
444219,effing cramps,0,1
1350979,@ShakeYourRump I'm good I've started my exam...,1,2
881582,@morriscat,1,1
5404,is starving but nothing looks appealing in my ...,0,0
1131959,movie night with the yearbook kidds i made am...,1,2
834026,Ouvindo Avril Lavigne - Let Go - I'm with You,1,1


In [48]:
from sklearn.model_selection import train_test_split

X = data.text.values
y = data.auto_sentiment.values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=10,stratify = data['auto_sentiment'])
y_train[0]

2

In [49]:

test_data.sample(5)


,text,sentiment,auto_sentiment
1349214,@medaox08 lol we should go to the movies today...,1,2
639734,Idk what to do anymore cryinnnn.,0,0
1448738,Me sporting my brand new glasses @melliebobel...,1,1
1372957,GoGoGo... move up by 2 102. ardhindie.com,1,1
936970,@EzaHY I hope that Sir Alex will do everythink...,1,2


In [50]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: GeForce GTX 1080 Ti


In [51]:
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords


def text_preprocessing(text):
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    text = strip_tags(text)
    text = strip_non_alphanum(text)
    text = strip_punctuation(text)
    text = strip_numeric(text)
    #text = remove_stopwords(text)
    text = text.lower()
    text = strip_multiple_whitespaces(text)
    #text = strip_short(text, minsize = 3)

    return text


[nltk_data] Downloading package stopwords to /home/felix/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess text
X_train_preprocessed = np.array([text_preprocessing(text) for text in X_train])
X_val_preprocessed = np.array([text_preprocessing(text) for text in X_val])


CPU times: user 18.1 s, sys: 256 ms, total: 18.4 s
Wall time: 18.4 s


In [53]:
X_train[38:39]

array(['@rialistic uhh go to settings then devices. '], dtype=object)

In [54]:
X_train_preprocessed[38]

' uhh go to settings then devices '

In [55]:
#preprocessing_for_bert([X_train_preprocessed[38]])

In [56]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

def get_auc_CV(model):
    """
    Return the average AUC score from cross-validation.
    """
    # Set KFold to shuffle data before the split
    kf = StratifiedKFold(5, shuffle=True, random_state=1)

    # Get AUC scores
    auc = cross_val_score(
        model, X_train_tfidf, y_train, scoring="roc_auc", cv=kf)

    return auc.mean()

In [58]:
from sklearn.metrics import accuracy_score, roc_curve, auc

def evaluate_roc(probs, y_true):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """
    preds = probs[:, 1]
    fpr, tpr, threshold = roc_curve(y_true, preds)
    roc_auc = auc(fpr, tpr)
    print(f'AUC: {roc_auc:.4f}')
       
    # Get accuracy over the test set
    y_pred = np.where(preds >= 0.5, 1, 0)
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy*100:.2f}%')
    
    # Plot ROC AUC
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [61]:
data[:1]['text']

1540228    LINES, VINES, AND TRYING TIMES CAME IN THE MAI...
Name: text, dtype: object

# BERT

In [62]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        #print("aaaa: ", tokenizer.convert_ids_to_tokens(encoded_sent.get('input_ids')))
        #print("sent: " , sent)
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [63]:
# Encode our concatenated data
encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in data.text.values]

# Find the maximum length
max_len = max([len(sent) for sent in encoded_tweets]) #230
print('Max length: ', max_len)

Max length:  230


In [64]:
# Specify `MAX_LEN`
MAX_LEN = max_len

# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
print('Original: ', X[0])
print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)


Original:  LINES, VINES, AND TRYING TIMES CAME IN THE MAILLLLLL  YAYYYYYYY :3 @jonasbrothers Love you guys &lt;3
Token IDs:  [101, 3210, 16702, 1998, 2667, 2335, 2234, 1999, 1996, 5653, 3363, 3363, 2140, 8038, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2293, 2017, 4364, 8318, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tokenizing data...


In [65]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [66]:
train_data

In [67]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 3

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: user 53 µs, sys: 0 ns, total: 53 µs
Wall time: 59.1 µs


In [68]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [75]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []
    true_label = []
    pred_label = []
    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()
        true_label.append(b_labels.cpu().numpy())
        pred_label.append(preds.cpu().numpy())
        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy, np.concatenate(true_label, axis=0 ),np.concatenate(pred_label, axis=0 )

In [70]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------


/home/felix/.local/lib/python3.8/site-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


   1    |   20    |   1.036229   |     -      |     -     |   12.83  
   1    |   40    |   0.872253   |     -      |     -     |   12.25  
   1    |   60    |   0.796319   |     -      |     -     |   12.20  
   1    |   80    |   0.710810   |     -      |     -     |   12.25  
   1    |   100   |   0.640099   |     -      |     -     |   12.27  
   1    |   120   |   0.520817   |     -      |     -     |   12.29  
   1    |   140   |   0.518902   |     -      |     -     |   12.30  
   1    |   160   |   0.502003   |     -      |     -     |   12.30  
   1    |   180   |   0.498359   |     -      |     -     |   12.33  
   1    |   200   |   0.434167   |     -      |     -     |   12.34  
   1    |   220   |   0.459249   |     -      |     -     |   12.32  
   1    |   240   |   0.453754   |     -      |     -     |   12.37  
   1    |   260   |   0.390624   |     -      |     -     |   12.35  
   1    |   280   |   0.405483   |     -      |     -     |   12.35  
   1    |   300   | 

   1    |  2380   |   0.248227   |     -      |     -     |   13.15  
   1    |  2400   |   0.243822   |     -      |     -     |   12.40  
   1    |  2420   |   0.190060   |     -      |     -     |   12.40  
   1    |  2440   |   0.260568   |     -      |     -     |   12.82  
   1    |  2460   |   0.247382   |     -      |     -     |   12.41  
   1    |  2480   |   0.227944   |     -      |     -     |   12.39  
   1    |  2500   |   0.277286   |     -      |     -     |   12.39  
   1    |  2520   |   0.233951   |     -      |     -     |   12.41  
   1    |  2540   |   0.230509   |     -      |     -     |   12.42  
   1    |  2560   |   0.272053   |     -      |     -     |   12.42  
   1    |  2580   |   0.250979   |     -      |     -     |   12.42  
   1    |  2600   |   0.228093   |     -      |     -     |   12.38  
   1    |  2620   |   0.248013   |     -      |     -     |   12.41  
   1    |  2640   |   0.284275   |     -      |     -     |   12.40  
   1    |  2660   | 

   1    |  4740   |   0.164777   |     -      |     -     |   12.30  
   1    |  4760   |   0.176457   |     -      |     -     |   12.31  
   1    |  4780   |   0.231332   |     -      |     -     |   12.32  
   1    |  4800   |   0.225550   |     -      |     -     |   12.32  
   1    |  4820   |   0.196428   |     -      |     -     |   12.32  
   1    |  4840   |   0.188156   |     -      |     -     |   12.33  
   1    |  4860   |   0.236997   |     -      |     -     |   12.32  
   1    |  4880   |   0.229965   |     -      |     -     |   12.31  
   1    |  4900   |   0.227999   |     -      |     -     |   12.33  
   1    |  4920   |   0.234634   |     -      |     -     |   12.33  
   1    |  4940   |   0.199255   |     -      |     -     |   12.28  
   1    |  4960   |   0.185481   |     -      |     -     |   12.31  
   1    |  4980   |   0.181011   |     -      |     -     |   12.33  
   1    |  5000   |   0.210975   |     -      |     -     |   12.32  
   1    |  5020   | 

...

   2    |  14700  |   0.156579   |     -      |     -     |   12.34  
   2    |  14720  |   0.132138   |     -      |     -     |   12.32  
   2    |  14740  |   0.133864   |     -      |     -     |   12.32  
   2    |  14760  |   0.134394   |     -      |     -     |   12.31  
   2    |  14780  |   0.140552   |     -      |     -     |   12.35  
   2    |  14800  |   0.125674   |     -      |     -     |   12.31  
   2    |  14820  |   0.119918   |     -      |     -     |   12.33  
   2    |  14840  |   0.123074   |     -      |     -     |   12.32  
   2    |  14860  |   0.089782   |     -      |     -     |   12.31  
   2    |  14880  |   0.138337   |     -      |     -     |   12.33  
   2    |  14900  |   0.130461   |     -      |     -     |   12.32  
   2    |  14920  |   0.101130   |     -      |     -     |   12.32  
   2    |  14940  |   0.087492   |     -      |     -     |   12.29  
   2    |  14960  |   0.130117   |     -      |     -     |   12.32  
   2    |  14980  | 

   2    |  17060  |   0.125171   |     -      |     -     |   12.30  
   2    |  17080  |   0.146161   |     -      |     -     |   12.33  
   2    |  17100  |   0.132379   |     -      |     -     |   12.31  
   2    |  17120  |   0.113878   |     -      |     -     |   12.30  
   2    |  17140  |   0.108435   |     -      |     -     |   12.32  
   2    |  17160  |   0.118854   |     -      |     -     |   12.31  
   2    |  17180  |   0.130353   |     -      |     -     |   12.30  
   2    |  17200  |   0.122871   |     -      |     -     |   12.32  
   2    |  17220  |   0.088503   |     -      |     -     |   12.32  
   2    |  17240  |   0.109726   |     -      |     -     |   12.33  
   2    |  17260  |   0.107786   |     -      |     -     |   12.32  
   2    |  17280  |   0.136970   |     -      |     -     |   12.30  
   2    |  17300  |   0.134541   |     -      |     -     |   12.31  
   2    |  17320  |   0.093364   |     -      |     -     |   12.30  
   2    |  17340  | 

   2    |  19420  |   0.080580   |     -      |     -     |   12.30  
   2    |  19440  |   0.120037   |     -      |     -     |   12.33  
   2    |  19460  |   0.127010   |     -      |     -     |   12.29  
   2    |  19480  |   0.111984   |     -      |     -     |   12.30  
   2    |  19500  |   0.116415   |     -      |     -     |   12.30  
   2    |  19520  |   0.136850   |     -      |     -     |   12.31  
   2    |  19540  |   0.104191   |     -      |     -     |   12.31  
   2    |  19560  |   0.155250   |     -      |     -     |   12.32  
   2    |  19580  |   0.138712   |     -      |     -     |   12.33  
   2    |  19600  |   0.142324   |     -      |     -     |   12.32  
   2    |  19620  |   0.132005   |     -      |     -     |   12.30  
   2    |  19640  |   0.098975   |     -      |     -     |   12.28  
   2    |  19660  |   0.119066   |     -      |     -     |   12.28  
   2    |  19680  |   0.160903   |     -      |     -     |   12.30  
   2    |  19700  | 

   2    |  21780  |   0.138259   |     -      |     -     |   12.31  
   2    |  21800  |   0.104014   |     -      |     -     |   12.31  
   2    |  21820  |   0.181075   |     -      |     -     |   12.31  
   2    |  21840  |   0.150525   |     -      |     -     |   12.30  
   2    |  21860  |   0.149330   |     -      |     -     |   12.32  
   2    |  21880  |   0.115913   |     -      |     -     |   12.30  
   2    |  21900  |   0.119955   |     -      |     -     |   12.29  
   2    |  21920  |   0.141332   |     -      |     -     |   12.30  
   2    |  21940  |   0.118273   |     -      |     -     |   12.31  
   2    |  21960  |   0.111076   |     -      |     -     |   12.28  
   2    |  21980  |   0.113108   |     -      |     -     |   12.29  
   2    |  22000  |   0.117540   |     -      |     -     |   12.33  
   2    |  22020  |   0.159331   |     -      |     -     |   12.31  
   2    |  22040  |   0.113221   |     -      |     -     |   12.30  
   2    |  22060  | 

   2    |  24140  |   0.160962   |     -      |     -     |   12.35  
   2    |  24160  |   0.112819   |     -      |     -     |   12.33  
   2    |  24180  |   0.121233   |     -      |     -     |   12.87  
   2    |  24200  |   0.135269   |     -      |     -     |   12.35  
   2    |  24220  |   0.134601   |     -      |     -     |   12.35  
   2    |  24240  |   0.120946   |     -      |     -     |   12.54  
   2    |  24260  |   0.094385   |     -      |     -     |   12.30  
   2    |  24280  |   0.132575   |     -      |     -     |   12.32  
   2    |  24300  |   0.132124   |     -      |     -     |   12.29  
   2    |  24320  |   0.126432   |     -      |     -     |   12.32  
   2    |  24340  |   0.102335   |     -      |     -     |   12.31  
   2    |  24360  |   0.107326   |     -      |     -     |   12.30  
   2    |  24380  |   0.148837   |     -      |     -     |   12.30  
   2    |  24400  |   0.116184   |     -      |     -     |   12.31  
   2    |  24420  | 

# Pickle DUMP

In [74]:
import pickle
with open('./bert_classifier3.pickle', 'wb') as handle:
    pickle.dump(bert_classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Pickle load

In [ ]:
import pickle
with open('./bert_classifier3.pickle', 'rb') as handle:
        bert_classifier = pickle.load(handle)

In [72]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [76]:
# Compute predicted probabilities on the test set

train_loss, train_accuracy,true_train_label,pred_train_label = evaluate(bert_classifier, train_dataloader)
val_loss, val_accuracy,true_val_label,pred_val_label = evaluate(bert_classifier, val_dataloader)


In [77]:

train_accuracy

97.09326729348557

In [78]:
val_accuracy

96.32509776039815

In [79]:
print(train_loss)
print(val_loss)

0.10388103589255578
0.13413253657578086


In [ ]:
# Run `preprocessing_for_bert` on the test set
#print('Tokenizing data...')
test_inputs, test_masks = preprocessing_for_bert(test_data.text)

# Create the DataLoader for our test set
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)
test_loss, test_accuracy = evaluate(bert_classifier, test_dataloader)
print(test_accuracy)

In [80]:
def sentiment_mapper(sentiment):
    if sentiment == 'negative':
        return 0
    if sentiment == 'neutral':
        return 1
    if sentiment == 'positive':
        return 2

In [81]:
air_data = pd.read_csv('./air_plane_tweets.csv')
air_data = air_data.sample(frac = 1) 
#air_data['auto_sentiment'] = air_data['auto_sentiment'].apply(sentiment_mapper)
air_data['auto_sentiment'] = air_data['airline_sentiment'].apply(sentiment_mapper)
#air_data['auto_sentiment'] = air_data['auto_sentiment'].map(lambda x: x+1)
X_air = air_data.text.values
y_air = air_data.auto_sentiment.values


#X_air_preprocessed = np.array([text_preprocessing(text) for text in X_air])
x_air_inputs, air_masks = preprocessing_for_bert(X_air)

air_labels = torch.tensor(y_air)

air_data = TensorDataset(x_air_inputs, air_masks, air_labels)
air_sampler = RandomSampler(air_data)
air_dataloader = DataLoader(air_data, sampler=air_sampler, batch_size=batch_size)

In [82]:
air_loss, air_accuracy,air_true_label,air_pred_label = evaluate(bert_classifier, air_dataloader)

In [ ]:
air_pred_label.tolist()

In [83]:
air_accuracy

55.499454148471614

In [ ]:
%matplotlib inline  
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report
import tensorflow as tf

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.metrics import precision_recall_fscore_support
#print(classification_report(test_labels, baseline_predicted))
cf = confusion_matrix(air_true_label.tolist(), air_pred_label.tolist(), labels=[0, 1, 2])

In [ ]:
df_cm = pd.DataFrame(cf, index = ["Negative", "Neutral", "Positive"],
                  columns = ["Negative", "Neutral", "Positive"])
plt.figure(figsize = (10,7))



sns.heatmap(df_cm/np.sum(df_cm), annot=True, 
            fmt='.2%', cmap='Blues')

In [ ]:
pr = precision_recall_fscore_support(air_true_label.tolist(), air_pred_label.tolist(), average='weighted')

In [ ]:
pr